In [1]:
import netCDF4 as nc
import sys, os, argparse
import time
import numpy as np
from numpy import ma as ma

#Import packages for plotting
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator
from pylab import imshow,cm
#Import packages for plotting
from matplotlib.colors import ListedColormap

#Import packages for clustering
from sklearn.cluster import KMeans
from scipy.linalg import eigh
from scipy.interpolate import LinearNDInterpolator as LNDI

#Import packages for geodesic distences
from pyproj import Geod

# Import package for parallel computing
from joblib import Parallel, delayed

from scipy.interpolate import griddata

import cartopy.crs as ccrs 
import cartopy.feature as cfeature  



In [2]:
parent_directory = "/cluster/home/llpui9007/Programs/HPC_Spectral_Clustering/"


# add utils folder to the TBarrier package
#sys.path.append(T_Barrier_directory+"/subfunctions/utils")
#sys.path.append(T_Barrier_directory+"/subfunctions/integration")
# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/Similarity_matrix_clustering")
sys.path.append(parent_directory+"/subfunctions/border_calculation")
sys.path.append(parent_directory+"/utils")


from ploters import ini_final_clusters
from ploters import gif_clusters
from ploters import ini_final_clusters_landmask
from ploters import gif_clusters_landmask
from calculating_borders import borders_binary

#Import packages for geodesic distences 
from pyproj import Geod

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/Similarity_matrix_construction")
sys.path.append(parent_directory+"/subfunctions/latlon_transform")
sys.path.append(parent_directory+"/utils")

#from trajectory_distance import integral_vec
#from similarity_matrix import similarity_matrix        #Commented out since they're currently in this script
from polar_rotation import polar_rotation_rx
from days_since_to_date import days_since_to_date


In [3]:
IC_resolution = 0.5
dt = 0.0025
DT = 0.01
freq = 1
e = 0
n_clusters = 20
# Format the variables
formatted_e = f"{e:.2f}"
formatted_DT = f"{DT:.4f}"
formatted_dt = f"{dt:.4f}"
# Define other necessary variables
year = 2009
season = "AMJ"
# Construct file paths and directories
Fmap_params = f"{year}_{season}_"
Fmap_params += f"ic{IC_resolution}_"
Fmap_params += f"dt{formatted_dt}_"
Fmap_params += f"DT{formatted_DT}"
directory =  f"/cluster/projects/nn8008k/lluisa/NextSIM/seas/" #f"/nird/projects/NS11048K/lluisa/NextSIM/rotated_ice_velocities/seas/AMJ/"
file_path = f"{directory}Fmap_10days/{Fmap_params}/"
parent_directory = "/cluster/home/llpui9007/Programs/HPC_Spectral_Clustering"
results_directory = file_path
regrided_geo_file_path = f"{directory}OPA-neXtSIM_CREG025_ILBOXE140_{year}_ice_90Rx_{season}_regrided.nc"
geo_file_path = f"{directory}OPA-neXtSIM_CREG025_ILBOXE140_{year}_ice_90Rx_{season}.nc"
K=1000
distance = 4
k_exp = 100

In [4]:
dataset = nc.Dataset(regrided_geo_file_path, mode='r')
#from m/s to m/day
siu = dataset.variables['vlon'][0,:,:]
land_mask_reg = dataset.variables['land_mask'][:,:]
# Access coordinates
latitude_reg = dataset.variables['regrided_rot_lat'][:]  
longitude_reg = dataset.variables['regrided_rot_lon'][:]
dataset.close()

dataset = nc.Dataset(geo_file_path, mode='r')
#from m/s to m/day
land_mask = dataset.variables['vlon'][0,:,:].mask
print("shape of land mask")
print(str(land_mask.shape))
# Access coordinates
latitude = dataset.variables['rot_lat'][:]  
longitude = dataset.variables['rot_lon'][:]
dataset.close()


shape of land mask
(492, 499)


In [5]:
# Define the 30 qualitative colors
colors = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
    "#aec7e8", "#ffbb78", "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f7b6d2", "#c7c7c7", "#dbdb8d", "#9edae5",
    "#393b79", "#637939", "#8c6d31", "#843c39", "#7b4173", "#5254a3", "#8ca252", "#bd9e39", "#ad494a", "#a55194"
]
# Create a ListedColormap
cmap = ListedColormap(colors, name='qualitative_30')
def plot_clusters_3w(Fmap, spars,n_clusters, labels_reweighted, labels_disp, labels, cmap, img_name, tmin): 
    # Define a diverging colormap and normalization centered at 1 with an asymmetric range
    #vlim_reweighted = max(abs(1-w_reweighted.min()),abs(1-w_reweighted[w_reweighted<w_reweighted.max()].max()))
    #vlim_disp = max(abs(1-w_disp.min()),abs(1-w_disp[w_disp<w_disp.max()].max()))
    IC = Fmap[0,:,:]
    IC_lat, IC_lon = polar_rotation_rx(IC[1], IC[0],-90) 
    positions_ini = np.asarray(np.vstack((IC_lon,IC_lat))) 

    # Create a figure with two subplots side by side
    fig, (ax1, ax2, ax3) = plt.subplots(
        1, 3,  # 1 row, 2 columns
        figsize=(24, 8),  # Adjust the figure size
        subplot_kw={"projection": ccrs.NorthPolarStereo()}  # North Polar Stereographic projection
    )

    # Define a color map for the clusters
    colors = plt.get_cmap(cmap, n_clusters)
    cluster_names = [f"Cluster {i+1}" for i in range(n_clusters)]  # Names for legend

    # Plot the first subplot with w_reweighted
    # Plot the initial distribution
    ax1.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_reweighted, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax1.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl1 = ax1.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl1.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl1.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax1.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax1.set_title("Re-weighted Lagrangian distance", fontsize=20)
  
    # Plot the second subplot with w_disp
    ax2.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_disp, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax2.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl1 = ax2.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl1.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl1.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax2.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax2.set_title("Dispersion measure", fontsize=20)

    # Plot the second subplot with w_vec
    ax3.scatter(positions_ini[0, :], positions_ini[1, :], c=labels, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax3.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl2 = ax3.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl2.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl2.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax3.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax3.set_title("Lagrangian distance ", fontsize=20)

    # Adjust layout and show the plot
    fig.suptitle("Nr clusters: "+str(n_clusters)+"    Sparsification parameter: "+str(spars)+"Km     tmin: "+str(tmin),fontsize=22)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.tight_layout()
    plt.show()
    plt.savefig(img_name, bbox_inches='tight')  # Save the figure if needed
    plt.close(fig)  # Close the figure to free up memory

In [6]:
# Define the 30 qualitative colors
colors = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
    "#aec7e8", "#ffbb78", "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f7b6d2", "#c7c7c7", "#dbdb8d", "#9edae5",
    "#393b79", "#637939", "#8c6d31", "#843c39", "#7b4173", "#5254a3", "#8ca252", "#bd9e39", "#ad494a", "#a55194"
]
# Create a ListedColormap
cmap = ListedColormap(colors, name='qualitative_30')
def plot_clusters_3w_ftle(Fmap, spars,n_clusters, labels_reweighted, labels_disp, labels, cmap, img_name, tmin): 
    # Define a diverging colormap and normalization centered at 1 with an asymmetric range
    #vlim_reweighted = max(abs(1-w_reweighted.min()),abs(1-w_reweighted[w_reweighted<w_reweighted.max()].max()))
    #vlim_disp = max(abs(1-w_disp.min()),abs(1-w_disp[w_disp<w_disp.max()].max()))

    tmin_idx=tmin*4
    tmax_idx=tmin_idx+10*4
    # Interpolate to a regular grid to then generate the interpolator objects.
    reg_vel_file_path = velocities_file_path[:-3]+'_regrided.nc'


    # Read dataset
    print("Reading regrided input data")
    dataset = nc.Dataset(reg_vel_file_path, mode='r')
    #from m/s to m/day
    interpolated_siu = dataset.variables['vlon'][tmin_idx:tmax_idx,:,:]
    interpolated_siv = dataset.variables['vlat'][tmin_idx:tmax_idx,:,:]
    interpolated_siu = np.transpose(interpolated_siu, axes=(1, 2, 0))
    interpolated_siv = np.transpose(interpolated_siv, axes=(1, 2, 0))
    regrided_land_mask = dataset.variables['land_mask'][:,:]
    # Access coordinates
    lat_grid = dataset.variables['regrided_rot_lat'][:]  
    lon_grid = dataset.variables['regrided_rot_lon'][:]
    # Access specific variables
    time_data = dataset.variables['time'][tmin_idx:tmax_idx] 
    time_data= np.reshape(time_data, (1,-1))
    dataset.close()

    lat_grid = lat_grid.filled()
    lon_grid = lon_grid.filled()

    lat_min, lat_max = lat_grid.min(), lat_grid.max()
    lon_min, lon_max = lon_grid.min(), lon_grid.max()

    x_domain = np.arange(lon_min, lon_max + dx, dx) # array (Nx, )
    y_domain = np.arange(lat_min, lat_max + dy, dy) # array (Ny, )

    X_domain, Y_domain = np.meshgrid(x_domain, y_domain) # array (Ny, Nx)


    ## Compute meshgrid of dataset
    X, Y =  lon_grid, lat_grid # array (NY, NX)

    ## Resolution of meshgrid
    dx_data = X[0,1]-X[0,0] # float
    dy_data = Y[1,0]-Y[0,0] # float

    delta = [dx_data, dy_data] # list (2, )


    # Initial time (in days)
    t0 = time_data[0,0] # float

    # Final time (in days)
    t1 = time_data[0,-1] # float

    # NOTE: For computing the backward trajectories: tN < t0 and dt < 0.

    cmap_ftle="seismic"
    vmin=-0.1
    vmax=0.1
    img_name=dir+'/'+str(tmin)+"_FTLE.png"

    FTLE = np.load(dir+str(tmin)+"_FTLE.npy")
    mask_interpolator = LNDI(list(zip(lat_grid.ravel(), lon_grid.ravel())), regrided_land_mask.ravel(),fill_value=1)
    mask=mask_interpolator(Y_domain,X_domain)
    #plotpolar_scatter_masked_ftle(X_domain, Y_domain, FTLE, ftle_land_mask, t0, tN,"seismic",-0.1,0.1, )


    mask = np.asarray(mask).astype(bool)
    Y_domain_rot, X_domain_rot = polar_rotation_rx(np.array(Y_domain),np.array(X_domain),-90) 
    IC = Fmap[0,:,:]
    IC_lat, IC_lon = polar_rotation_rx(IC[1], IC[0],-90) 
    positions_ini = np.asarray(np.vstack((IC_lon,IC_lat))) 

    # Create a figure with two subplots side by side
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(
        1, 4,  # 1 row, 2 columns
        figsize=(32, 8),  # Adjust the figure size
        subplot_kw={"projection": ccrs.NorthPolarStereo()}  # North Polar Stereographic projection
    )

    # Define a color map for the clusters
    colors = plt.get_cmap(cmap, n_clusters)
    cluster_names = [f"Cluster {i+1}" for i in range(n_clusters)]  # Names for legend

    # Plot the first subplot with w_reweighted
    # Plot the initial distribution
    ax1.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_reweighted, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax1.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl1 = ax1.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl1.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl1.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax1.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax1.set_title("Re-weighted Lagrangian distance", fontsize=20)
  
    # Plot the second subplot with w_disp
    ax2.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_disp, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax2.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl1 = ax2.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl1.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl1.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax2.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax2.set_title("Dispersion measure", fontsize=20)

    # Plot the second subplot with w_vec
    ax3.scatter(positions_ini[0, :], positions_ini[1, :], c=labels, cmap=colors, vmin=0, vmax=n_clusters-1,transform=ccrs.PlateCarree(), s=8)
    ax3.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl2 = ax3.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl2.xlocator = plt.MultipleLocator(45)  # Longitude gridlines every 45 degrees
    gl2.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    ax3.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax3.set_title("Lagrangian distance ", fontsize=20)

    # Choose a colormap (e.g., 'viridis')
    masked = np.array(mask.ravel())
    cax = ax4.scatter(np.asarray(X_domain_rot.ravel())[0,~masked], np.asarray(Y_domain_rot.ravel())[0,~masked], c= np.asarray(FTLE.ravel())[~masked], cmap= cmap_ftle,transform=ccrs.PlateCarree(), s=8,vmin=vmin,vmax=vmax)
    #cbar = fig.colorbar(cax, ticks = np.linspace(0, .4, 9))
    #cbar.set_label('FTLE [$\mathrm{days^{-1}}$]', fontsize=18)
    #cbar.tick_params(labelsize=14)  # Adjust '14' to your desired tick size
    #cbar.set_ticks([vmin, vmin/2, 0, vmax/2, vmax])

    # Add coastlines and gridlines 
    ax4.coastlines(resolution='50m', color='black', linewidth=0.8) 
    gl4 = ax4.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5) 
    gl4.ylocator = plt.MultipleLocator(35)  # Latitude gridlines every 35 degrees
    # Set the extent to focus on the North Pole 
    ax4.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree()) 
    # Add a title 
    ax4.set_title(r'$ \mathrm{FTLE}$'+f'$_{{{days_since_to_date(t0)}}}^{{{days_since_to_date(t1)}}}$', fontsize = 24) 
    # Save and show the plot 


    # Adjust layout and show the plot
    fig.suptitle("Nr clusters: "+str(n_clusters)+"    Sparsification parameter: "+str(spars)+"Km     tmin: "+str(tmin),fontsize=22)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.tight_layout()
    plt.show()
    plt.savefig(img_name, bbox_inches='tight')  # Save the figure if needed
    plt.close(fig)  # Close the figure to free up memory





In [7]:
# Interpolate to a regular grid to then generate the interpolator objects
filename = f"OPA-neXtSIM_CREG025_ILBOXE140_{year}_ice_90Rx_{season}.nc"
directory = "/cluster/projects/nn8008k/lluisa/NextSIM/seas/"
velocities_file_path = os.path.join(directory, filename)
reg_vel_file_path = velocities_file_path[:-3] + '_regrided.nc'
# Read dataset
print("Reading regrided input data")
dataset = nc.Dataset(reg_vel_file_path, mode='r')
regrided_land_mask = dataset.variables['land_mask'][:, :]
lat_grid = dataset.variables['regrided_rot_lat'][:]
lon_grid = dataset.variables['regrided_rot_lon'][:]
time_data = dataset.variables['time'][:]
time_data = np.reshape(time_data, (1, -1))
dataset.close()
lat_grid = lat_grid.filled()
lon_grid = lon_grid.filled()
lat_min, lat_max = lat_grid.min(), lat_grid.max()
lon_min, lon_max = lon_grid.min(), lon_grid.max()
dx = 0.1 # float
dy = 0.1 # float
x_domain = np.arange(lon_min, lon_max + dx, dx)
y_domain = np.arange(lat_min, lat_max + dy, dy)
X_domain, Y_domain = np.meshgrid(x_domain, y_domain)
X, Y = lon_grid, lat_grid
dx_data = X[0, 1] - X[0, 0]
dy_data = Y[1, 0] - Y[0, 0]
delta = [dx_data, dy_data]
mask_interpolator = LNDI(list(zip(lat_grid.ravel(), lon_grid.ravel())), regrided_land_mask.ravel(), fill_value=1)
mask = mask_interpolator(Y_domain, X_domain)
mask = np.asarray(mask).astype(bool)
Y_domain_rot, X_domain_rot = polar_rotation_rx(np.array(Y_domain), np.array(X_domain), -90)

Reading regrided input data


In [8]:
def plot_clusters_3w_ftle(Fmap, spars, n_clusters, labels_reweighted, labels_disp, labels, cmap, img_name, tmin, time_data,X_domain_rot,Y_domain_rot,mask,ftle_dir):
    # Define a diverging colormap and normalization centered at 1 with an asymmetric range
    tmin_idx = tmin * 4
    tmax_idx = tmin_idx + 10 * 4
    t0 = time_data[0, tmin_idx]
    t1 = time_data[0, tmax_idx]
    cmap_ftle = "seismic"
    vmin = -0.1
    vmax = 0.1
    FTLE = np.load(ftle_dir + str(tmin) + "_FTLE.npy")
    IC = Fmap[0, :, :]
    IC_lat, IC_lon = polar_rotation_rx(IC[1], IC[0], -90)
    positions_ini = np.asarray(np.vstack((IC_lon, IC_lat)))
    # Create a figure with four subplots side by side
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(
        1, 4,  # 1 row, 4 columns
        figsize=(32, 8),  # Adjust the figure size
        subplot_kw={"projection": ccrs.NorthPolarStereo()}  # North Polar Stereographic projection
    )
    # Define a color map for the clusters
    colors = plt.get_cmap(cmap, n_clusters)
    # Plot the first subplot with w_reweighted
    ax1.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_reweighted, cmap=colors, vmin=0, vmax=n_clusters - 1,
                transform=ccrs.PlateCarree(), s=8)
    ax1.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl1 = ax1.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl1.xlocator = plt.MultipleLocator(45)
    gl1.ylocator = plt.MultipleLocator(35)
    ax1.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax1.set_title("Re-weighted Lagrangian distance", fontsize=20)
    # Plot the second subplot with w_disp
    ax2.scatter(positions_ini[0, :], positions_ini[1, :], c=labels_disp, cmap=colors, vmin=0, vmax=n_clusters - 1,
                transform=ccrs.PlateCarree(), s=8)
    ax2.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl2 = ax2.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl2.xlocator = plt.MultipleLocator(45)
    gl2.ylocator = plt.MultipleLocator(35)
    ax2.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax2.set_title("Dispersion measure", fontsize=20)
    # Plot the third subplot with w_vec
    ax3.scatter(positions_ini[0, :], positions_ini[1, :], c=labels, cmap=colors, vmin=0, vmax=n_clusters - 1,
                transform=ccrs.PlateCarree(), s=8)
    ax3.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl3 = ax3.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl3.xlocator = plt.MultipleLocator(45)
    gl3.ylocator = plt.MultipleLocator(35)
    ax3.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax3.set_title("Lagrangian distance", fontsize=20)
    # Plot the FTLE data in ax4
    masked = np.array(mask.ravel())
    cax = ax4.scatter(
        np.asarray(X_domain_rot.ravel())[0, ~masked],
        np.asarray(Y_domain_rot.ravel())[0, ~masked],
        c=np.asarray(FTLE.ravel())[~masked],
        cmap=cmap_ftle,
        transform=ccrs.PlateCarree(),
        s=8,
        vmin=vmin,
        vmax=vmax
    )
    ax4.coastlines(resolution='50m', color='black', linewidth=0.8)
    gl4 = ax4.gridlines(draw_labels=False, color='gray', linestyle='--', linewidth=0.5)
    gl4.ylocator = plt.MultipleLocator(35)
    ax4.set_extent([-180, 180, 65, 90], crs=ccrs.PlateCarree())
    ax4.set_title(r'$ \mathrm{FTLE}$'+f'$_{{{days_since_to_date(t0)}}}^{{{days_since_to_date(t1)}}}$', fontsize=24)
    # Add a separate axis for the colorbar
    cbar_ax = fig.add_axes([0.9, 0.05, 0.01, 0.8])  # [left, bottom, width, height]
    cbar = fig.colorbar(cax, cax=cbar_ax, ticks=np.linspace(vmin, vmax, 5))
    cbar.set_label('FTLE [$\mathrm{days^{-1}}$]', fontsize=18)
    cbar.ax.tick_params(labelsize=14)
    # Adjust layout and show the plot
    fig.suptitle("Nr clusters: "+str(n_clusters)+"    Sparsification parameter: "+str(spars)+"Km     tmin: "+str(tmin)+f'$, t_0={{{days_since_to_date(t0)}}},  t_1={{{days_since_to_date(t1)}}}$', fontsize=22)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to leave space for the colorbar
    plt.savefig(img_name, bbox_inches='tight')  # Save the figure if needed
    #plt.show()
    plt.close(fig)  # Close the figure to free up memory

In [9]:
for tmin in range(58,81):
    d=0
    n_clusters=15
    i=n_clusters


    formatted_d = f"{d:.2f}"

    clusters_dir="/cluster/projects/nn8008k/lluisa/NextSIM/seas/Fmap_10days/"+str(year)+"_"+season+"_ic0.5_dt0.0025_DT0.0100/clusters_K10000000_border4.00_center_spars_"+formatted_d+"/"
    labels_path = clusters_dir+str(i)+"_tmin"+str(tmin)+"_spar"+str(formatted_d)+"_labels.npy"
    labels=np.load(labels_path)
    labels_disp_path = clusters_dir+str(i)+"_tmin"+str(tmin)+"_spar"+str(formatted_d)+"_labels_disp.npy"
    labels_disp=np.load(labels_disp_path)
    labels_reweighted_path = clusters_dir+str(i)+"_tmin"+str(tmin)+"_spar"+str(formatted_d)+"_labels_reweighted.npy"
    labels_reweighted=np.load(labels_reweighted_path)
    Fmap_path = clusters_dir+"tmin"+str(tmin)+"_spar"+str(formatted_d)+"_labels_Fmap_cut.npy"
    Fmap=np.load(Fmap_path)
    filename = f"OPA-neXtSIM_CREG025_ILBOXE140_{year}_ice_90Rx_{season}.nc"
    directory = "/cluster/projects/nn8008k/lluisa/NextSIM/seas/"
    velocities_file_path = os.path.join(directory, filename)
    dx = 0.1 # float
    dy = 0.1 # float
    ftle_dir = "/cluster/projects/nn8008k/lluisa/NextSIM/seas/Fmap_10days/cleaned_FTLE_"+str(year)+"_"+season+"_dx0.100_dy0.100_dt0.100_grid_ratio0.010/"
    output=clusters_dir+str(i)+"_tmin"+str(tmin)+"_spar"+str(formatted_d)+".png"

    plot_clusters_3w_ftle(Fmap, d, n_clusters, labels_reweighted, labels_disp, labels, cmap,output,tmin,time_data,X_domain_rot,Y_domain_rot,mask,ftle_dir) #clusters_dir+str(20)+"clusters_tmin0.png")

/tmp/ipykernel_1542302/385032305.py:73: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to leave space for the colorbar
/tmp/ipykernel_1542302/385032305.py:73: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to leave space for the colorbar
/tmp/ipykernel_1542302/385032305.py:73: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to leave space for the colorbar
/tmp/ipykernel_1542302/385032305.py:73: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Adjust layout to leave space for the colorbar
/tmp/ipykernel_15423